In [7]:
import os
import glob
import numpy as np
import mne
import xml.etree.ElementTree as ET
from scipy.signal import butter, filtfilt, iirnotch, welch
from joblib import Parallel, delayed

In [8]:
# Configuration
BASE_DIR = "../../data/MESA"
EDF_DIR = os.path.join(BASE_DIR, 'edfs')
ANNOTATION_DIR = os.path.join(BASE_DIR, 'mesa_200_annotations-events-nssr')
OUTPUT_DIR = '../../processed_mesa'
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [9]:
# --- Locate all EDF files across multiple subdirectories ---
# List of directory names containing EDFs under BASE_DIR
edf_subdir_names = ['mesa_200_subset_edfs'] + [f'mesa_200_subset_edfs {i}' for i in range(2, 19)]
all_edf_files = []
for subdir_name in edf_subdir_names:
    dir_path = os.path.join(EDF_DIR, subdir_name)
    if os.path.isdir(dir_path):
        edf_files_in_subdir = glob.glob(os.path.join(dir_path, '*.edf')) + \
                              glob.glob(os.path.join(dir_path, '*.EDF'))
        all_edf_files.extend(edf_files_in_subdir)
    else:
        print(f"Warning: Directory not found - {dir_path}")

# Remove duplicates if any file exists in multiple lists and sort
all_edf_files = sorted(list(set(all_edf_files)))

print(f"Found {len(all_edf_files)} EDF files to process across specified directories.")
# --- End of EDF file location ---

Found 200 EDF files to process across specified directories.


In [10]:
# List of already-processed PSG recording IDs (without .edf)
PROCESSED_ALREADY = [
    "mesa-sleep-0167",
    "mesa-sleep-0359",
    "mesa-sleep-0629",
    "mesa-sleep-0601",
    "mesa-sleep-0211",
    "mesa-sleep-0563",
    "mesa-sleep-0204",
    "mesa-sleep-0589",
    "mesa-sleep-0006",
    "mesa-sleep-0012",
    "mesa-sleep-0628",
    "mesa-sleep-0614",
    "mesa-sleep-0402",
    "mesa-sleep-0399",
    "mesa-sleep-0416",
    "mesa-sleep-0414",
    "mesa-sleep-0372",
    "mesa-sleep-0602",
    "mesa-sleep-0038",
    "mesa-sleep-0548",
    "mesa-sleep-0561",
    "mesa-sleep-0159",
    "mesa-sleep-0171",
    "mesa-sleep-0388",
    "mesa-sleep-0415",
    "mesa-sleep-0363",
    "mesa-sleep-0149",
    "mesa-sleep-0001",
    "mesa-sleep-0565",
    "mesa-sleep-0558",
    "mesa-sleep-0014",
    "mesa-sleep-0028",
    "mesa-sleep-0612",
    "mesa-sleep-0604",
    "mesa-sleep-0002",
    "mesa-sleep-0016",
    "mesa-sleep-0572",
    "mesa-sleep-0599",
    "mesa-sleep-0474",
    "mesa-sleep-0188",
    "mesa-sleep-0312",
    "mesa-sleep-0460",
    "mesa-sleep-0306",
    "mesa-sleep-0138",
    "mesa-sleep-0110",
    "mesa-sleep-0064",
    "mesa-sleep-0070",
]


In [11]:
# EEG 1 is the same as EEG Fz-Cz
# EEG 2 is the same as EEG Cz-Oz
# EEG 3 is the same as EEG C4 M1
# Kim's source: https://sleepdata.org/datasets/mesa/pages/equipment/montage-and-sampling-rate-information.md 

CHANNELS_TO_LOAD = ["EEG 1", "EOG L", "EOG R"]  # Adjust based on available channels
TARGET_SFREQ = 256.0
LOW_FREQ = 0.5
HIGH_FREQ = 30.0
EPOCH_LENGTH = 30.0
SEQ_LENGTH = 20
SEQ_STRIDE = 10

# Sleep stage mapping based on NSRR annotations
ANNOTATION_MAP = {
    "0": 0,  # Wake
    "1": 1,  # N1
    "2": 2,  # N2
    "3": 3,  # N3
    "5": 4   # REM
}

def auto_notch_filter(signal, fs, center_guess=60.0, search_range=1.0, quality=30.0):
    freqs, psd = welch(signal, fs=fs, nperseg=2048)
    idx = np.where((freqs >= center_guess - search_range) & (freqs <= center_guess + search_range))[0]
    if len(idx) == 0:
        return signal
    true_freq = freqs[idx[np.argmax(psd[idx])]]
    w0 = true_freq / (0.5 * fs)
    b, a = iirnotch(w0, quality)
    return filtfilt(b, a, signal)

def parse_xml_annotations(xml_path):
    """Parse the XML file to extract sleep stage annotations."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    stages = []
    for event in root.findall(".//ScoredEvent"):
        if event.find("EventType").text == "Stages|Stages":
            concept = event.find("EventConcept").text.strip().split("|")[-1]  # Get the last part of the stage name
            if concept not in ANNOTATION_MAP:
                continue
            stage_id = ANNOTATION_MAP[concept]
            start = float(event.find("Start").text)
            duration = float(event.find("Duration").text)

            # Split duration into multiple 30-second epochs
            num_epochs = int(duration // 30)
            for i in range(num_epochs):
                epoch_start = start + i * 30
                stages.append((epoch_start, 30.0, stage_id))
    
    return stages

def process_record(psg_path, xml_path, channels, target_sfreq, low_freq, high_freq, epoch_length):
    """Process a single PSG file with its corresponding XML annotations."""
    raw = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)
    if raw.info['sfreq'] != target_sfreq:
        raw.resample(target_sfreq, npad="auto", verbose=False)
    picks = mne.pick_types(raw.info, eeg=True, eog=True)
    if len(picks) < len(channels):
        raise ValueError(f"Not all requested channels found in {psg_path}: {channels}")
    raw.filter(l_freq=low_freq, h_freq=high_freq, picks=picks, verbose=False)

    # Apply notch to each picked channel individually (MNE doesn't support adaptive notch out-of-the-box)
    for i, ch in enumerate(picks):
        signal = raw.get_data(picks=[ch])[0]
        filtered = auto_notch_filter(signal, fs=raw.info['sfreq'], center_guess=60.0)
        raw._data[ch] = filtered

    annotations = parse_xml_annotations(xml_path)

    # Convert annotations to MNE-compatible events
    events = []
    for start, duration, stage_id in annotations:
        sample = int(start * raw.info['sfreq'])
        events.append([sample, 0, stage_id])

    events = np.array(events)
    tmin = 0.0
    tmax = epoch_length - 1 / raw.info['sfreq']
    epochs = mne.Epochs(raw, events=events, event_id=ANNOTATION_MAP, tmin=tmin, tmax=tmax,
                        baseline=None, preload=True, verbose=False)
    data = epochs.get_data()
    labels = epochs.events[:, -1]

    for ch in range(data.shape[1]):
        m = np.mean(data[:, ch, :])
        s = np.std(data[:, ch, :]) if np.std(data[:, ch, :]) != 0 else 1.0
        data[:, ch, :] = (data[:, ch, :] - m) / s
    return data, labels, raw.ch_names

def create_sequences(data, labels, seq_length, seq_stride):
    """Create sequences of epochs."""
    n_epochs = data.shape[0]
    sequences, seq_labels = [], []
    for start in range(0, n_epochs - seq_length + 1, seq_stride):
        sequences.append(data[start:start+seq_length])
        seq_labels.append(labels[start:start+seq_length])
    return np.array(sequences), np.array(seq_labels)

def process_and_save(psg_file, output_dir, channels):
    """Process and save data for a single PSG file."""
    rec_id = os.path.basename(psg_file).replace(".edf", "")
    
    if rec_id in PROCESSED_ALREADY:
        print(f"Skipping {rec_id}: already processed.")
        return

    xml_file = os.path.join(ANNOTATION_DIR, rec_id + "-nsrr.xml")
    if not os.path.exists(xml_file):
        print(f"Annotation file not found for {psg_file}, skipping.")
        return
    try:
        data, labels, ch_names = process_record(psg_file, xml_file, channels,
                                                TARGET_SFREQ, LOW_FREQ, HIGH_FREQ, EPOCH_LENGTH)
        
        if data.shape[0] == 0 or labels.shape[0] == 0:
            print(f"No valid epochs for {psg_file}, skipping.")
            return

    except Exception as e:
        print(f"Error processing {psg_file}: {e}")
        return

    try:
        np.savez_compressed(os.path.join(output_dir, f"{rec_id}_epochs.npz"),
                            data=data.astype('float32'), labels=labels.astype('int8'))
        sequences, seq_labels = create_sequences(data, labels, SEQ_LENGTH, SEQ_STRIDE)
        np.savez_compressed(os.path.join(output_dir, f"{rec_id}_sequences.npz"),
                            sequences=sequences.astype('float32'), seq_labels=seq_labels.astype('int8'))
        print(f"Processed {rec_id}: epochs {data.shape[0]}, sequences {sequences.shape[0]}, channels: {ch_names}")
    except Exception as e:
        print(f"Error saving {psg_file}: {e}")
        return

In [ ]:
def main():
    psg_files = glob.glob(os.path.join(EDF_DIR, "*.edf"))
    print(f"Found {len(psg_files)} PSG files.")
    Parallel(n_jobs=2)(delayed(process_and_save)(f, OUTPUT_DIR, CHANNELS_TO_LOAD) for f in psg_files)

if __name__ == '__main__':
    main()

Found 200 PSG files.
Skipping mesa-sleep-0167: already processed.
Skipping mesa-sleep-0359: already processed.
Skipping mesa-sleep-0601: already processed.
Skipping mesa-sleep-0629: already processed.
Skipping mesa-sleep-0563: already processed.
Skipping mesa-sleep-0211: already processed.
Skipping mesa-sleep-0204: already processed.
Skipping mesa-sleep-0589: already processed.
Skipping mesa-sleep-0012: already processed.
Skipping mesa-sleep-0006: already processed.
Skipping mesa-sleep-0628: already processed.
Skipping mesa-sleep-0614: already processed.
Skipping mesa-sleep-0402: already processed.
Skipping mesa-sleep-0416: already processed.
Skipping mesa-sleep-0399: already processed.
Skipping mesa-sleep-0414: already processed.
Skipping mesa-sleep-0372: already processed.
Skipping mesa-sleep-0602: already processed.
Error processing ../../data/MESA/edfs/mesa-sleep-0560.edf: No matching events found for 5 (event id 4)
Skipping mesa-sleep-0548: already processed.
Skipping mesa-sleep-0